In [1]:
import pandas as pd
from os import listdir
import json

import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score

from nltk.tokenize import word_tokenize

import fasttext

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/luungoc2005/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
model = fasttext.load_model('data/cc.en.300.bin')

In [3]:

def tokenize_text_list(input_list):
    text_list = [word_tokenize(' '.join(text.split())) for text in input_list]
    return text_list


In [4]:
def get_sentence_vectors(text_list):
    vectors = [
        np.average(
            np.array([model.get_word_vector(word) for word in sentence])
        , axis=0)
        for sentence in text_list
        if len(sentence) > 0
    ]
    return vectors

In [16]:
with open('data/kc_data.json', 'r') as input_file:
    training_file = json.load(input_file)

# extract raw intents and labels from file. Needs upgrade
raw_examples = []
raw_labels = []

for intent in training_file:
    for usersay in intent['usersays']:
        raw_labels.append(intent['name'])
        raw_examples.append(usersay)

raw_exampes_tokens = tokenize_text_list(raw_examples)

le = LabelEncoder()
X_train = np.array(get_sentence_vectors(raw_exampes_tokens))
y_train = np.array(le.fit_transform(raw_labels))

clf = MLPClassifier(
    hidden_layer_sizes=(50,), 
    random_state=1,
    batch_size=min(64, len(X_train)),
    max_iter=100,
    verbose=True)

clf.fit(X_train, y_train)

preds = clf.predict(X_train)


Iteration 1, loss = 7.14182879
Iteration 2, loss = 6.79876860
Iteration 3, loss = 6.50328011
Iteration 4, loss = 6.18189136
Iteration 5, loss = 5.86140997
Iteration 6, loss = 5.56002529
Iteration 7, loss = 5.28511496
Iteration 8, loss = 5.03376433
Iteration 9, loss = 4.80442449
Iteration 10, loss = 4.58962182
Iteration 11, loss = 4.39183168
Iteration 12, loss = 4.20726391
Iteration 13, loss = 4.03151946
Iteration 14, loss = 3.86788594
Iteration 15, loss = 3.71592058
Iteration 16, loss = 3.57443422
Iteration 17, loss = 3.43849237
Iteration 18, loss = 3.30976726
Iteration 19, loss = 3.19442013
Iteration 20, loss = 3.08565398
Iteration 21, loss = 2.98549509
Iteration 22, loss = 2.88924336
Iteration 23, loss = 2.79792789
Iteration 24, loss = 2.71630624
Iteration 25, loss = 2.63498838
Iteration 26, loss = 2.56082108
Iteration 27, loss = 2.48897167
Iteration 28, loss = 2.42227378
Iteration 29, loss = 2.35537274
Iteration 30, loss = 2.29452072
Iteration 31, loss = 2.23759425
Iteration 32, los

/Users/luungoc2005/miniconda3/envs/vn-nlp/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [17]:
print(f'Accuracy: {accuracy_score(y_train, preds)}')
print(f'F1: {f1_score(y_train, preds, average="weighted")}')
# print(f'Accuracy: {accuracy_score(y_train, preds)}')

Accuracy: 0.8635920436817472
F1: 0.8614763638732915


/Users/luungoc2005/miniconda3/envs/vn-nlp/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
